In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType,TimestampType
from pyspark.sql.types import DateType
from pyspark.sql.functions import *

#pip install findspark
#/usr/local/opt/apache-spark/libexec/sbin
#./start-thriftserver.sh

In [2]:
import findspark
findspark.init()

In [5]:
spark = SparkSession\
    .builder\
    .appName("spark")\
    .config('spark.driver.memory', '32g')\
    .config("hive.server2.thrift.port", 10000)\
    .config("spark.sql.hive.thriftServer.singleSession", True)\
    .enableHiveSupport()\
    .getOrCreate()


In [8]:
schema = StructType([
    StructField("ID", IntegerType()),
    StructField("Speed", DoubleType()),
    StructField("TravelTime", IntegerType()),
    StructField("Status", IntegerType()),
    StructField("timedate", StringType()),
    StructField("LinkId", IntegerType()),
    StructField("LinkPoints", StringType()),
    StructField("EncodedLinkPoints", StringType()),
    StructField("EncodedPolyLineLvls", StringType()),
    StructField("Owner", StringType()),
    StructField("TranscomId", IntegerType()),
    StructField("Borough", StringType()),
    StructField("Link_Name", StringType())
])

schema1 = "ID bigint, Speed float, TravelTime int, Status int, TimeDate string, LinkId int, LinkPoints string, EncodedLinkPoints string, EncodedPolyLineLvls string, Owner string, TranscomId int, Borough string,Link_Name string"
df2 = spark.read.format("csv").option("header", "true").schema(schema).load("file:/Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv")
df2 = df2.withColumn("timedate", to_timestamp("timedate", 'MM/dd/yyyy hh:mm:ss a'))
df2 = df2.na.drop()
df2 = df2.drop(*("Status","EncodedLinkPoints","EncodedPolyLineLvls","TranscomId"))
df2.printSchema()
df2.createOrReplaceTempView("dataset")
# .option("inferSchema", "true")

root
 |-- ID: integer (nullable = true)
 |-- Speed: double (nullable = true)
 |-- TravelTime: integer (nullable = true)
 |-- timedate: timestamp (nullable = true)
 |-- LinkId: integer (nullable = true)
 |-- LinkPoints: string (nullable = true)
 |-- Owner: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Link_Name: string (nullable = true)



In [18]:
data2022 = spark.sql("select LinkId, split(LinkPoints,' ') as `Link Points`, count(LinkPoints) from dataset group by LinkId,`Link Points` order by LinkId asc")
display(data2022)

DataFrame[LinkId: int, Link Points: array<string>, count(LinkPoints): bigint]

In [22]:
data2022.collect()

22/03/24 21:02:05 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ID, SPEED, TRAVEL_TIME, STATUS, DATA_AS_OF, LINK_ID, LINK_POINTS, ENCODED_POLY_LINE, ENCODED_POLY_LINE_LVLS, OWNER, TRANSCOM_ID, BOROUGH, LINK_NAME
 Schema: ID, Speed, TravelTime, Status, timedate, LinkId, LinkPoints, EncodedLinkPoints, EncodedPolyLineLvls, Owner, TranscomId, Borough, Link_Name
Expected: TravelTime but found: TRAVEL_TIME
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv


[Row(LinkId=4329472, Link Points=['40.75829,-73.997531', '40.7605,-74.0032', '40.7620605,-74.007021', '40.7662905,-74.01706'], count(LinkPoints)=433145),
 Row(LinkId=4329473, Link Points=['40.7578106,-73.996801', '40.7604506,-74.003221', '40.762021,-74.00705', '40.76621,-74.01712'], count(LinkPoints)=431331),
 Row(LinkId=4329480, Link Points=['40.6211706,-74.16874', '40.6213905,-74.16988', '40.62163,-74.171911', '40.6219705,-74.173431', '40.6222604,-74.17418', '40.6235,-74.175451', '40.6257105,-74.17843', '40.62641,-74.1794', '40.62711,-74.18058', '40.62834,-74.182701', '40.6297206,-74.18544', '40.63696,-74.199131'], count(LinkPoints)=2),
 Row(LinkId=4329481, Link Points=['40.61688,-74.17731', '40.6189004,-74.17604', '40.6204505,-74.17537', '40.6219004,-74.17519', '40.623321,-74.175361', '40.62406,-74.17586', '40.62456,-74.17668', '40.6251,-74.177771'], count(LinkPoints)=2),
 Row(LinkId=4329483, Link Points=['40.5264504,-74.27001', '40.52568,-74.267851', '40.52528,-74.26585', '40.52492

In [39]:
data2022.write.format('json').mode('overwrite').save('file:/Users/minoseah629/Desktop/ait614/linkpoints by id.json')


22/03/23 22:35:33 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ID, SPEED, TRAVEL_TIME, STATUS, DATA_AS_OF, LINK_ID, LINK_POINTS, ENCODED_POLY_LINE, ENCODED_POLY_LINE_LVLS, OWNER, TRANSCOM_ID, BOROUGH, LINK_NAME
 Schema: ID, Speed, TravelTime, Status, timedate, LinkId, LinkPoints, EncodedLinkPoints, EncodedPolyLineLvls, Owner, TranscomId, Borough, Link_Name
Expected: TravelTime but found: TRAVEL_TIME
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv


In [6]:
data2022.count()

22/03/22 23:33:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ID, SPEED, TRAVEL_TIME, STATUS, DATA_AS_OF, LINK_ID, LINK_POINTS, ENCODED_POLY_LINE, ENCODED_POLY_LINE_LVLS, OWNER, TRANSCOM_ID, BOROUGH, LINK_NAME
 Schema: ID, Speed, TravelTime, Status, timedate, LinkId, LinkPoints, EncodedLinkPoints, EncodedPolyLineLvls, Owner, TranscomId, Borough, Link_Name
Expected: TravelTime but found: TRAVEL_TIME
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv


1893262

In [1]:
spark.stop()

NameError: name 'spark' is not defined